In [ ]:
from dotenv import load_dotenv
import os
import base64
from io import StringIO
import snowflake.connector as sf
from dotenv import load_dotenv
import json
import pandas as pd
load_dotenv()
from e2b_code_interpreter import Sandbox

In [ ]:
def sf_client():
    conn = sf.connect(
    user=os.getenv('SF_USER'),
    password=os.getenv('SF_PASSWORD'),
    account=os.getenv('SF_ACCOUNT'),
    warehouse=os.getenv('SF_WAREHOUSE'),
    database=os.getenv('SF_DATABASE'),
    schema=os.getenv('SF_SCHEMA'),
    role='FRED_ROLE',
    private_key_file = 'rsa_key.p8'
    )
    return conn
 
conn = sf_client()
cursor = conn.cursor()

In [ ]:
df = cursor.execute('select * from STOCK_DATA').fetch_pandas_all()

In [ ]:
os.makedirs('local', exist_ok=True)
df.to_csv('local/data.csv', index=False)

### LLM SQL Generation

In [ ]:
from utils.snowflake.core import write_to_csv
from utils.helper import sql_query_generation_prompt
from utils.litellm.core import llm
import json 

result = llm(model='gemini/gemini-1.5-pro', system_prompt=sql_query_generation_prompt, user_prompt='generate 5 sql queries', is_json=True)
print(result)
answer = json.loads(result['answer']) if isinstance(result['answer'], str) else result['answer']

In [ ]:
with open('local/charts.json', 'w') as f:
    f.write(json.dumps(answer))

In [ ]:
# sqls = []
# for _ in answer:
#     sqls.append(_['SQL']+';')
# with open('local/charts.sql', 'w') as f:
#     f.write("\n".join(sqls))

### SQL (Snowflake) -> CSV

In [ ]:
# from utils.snowflake.core import write_to_csv

# with open('local/charts.sql', 'r') as f:
#     sql_content = f.read()
    
# for idx, sql in enumerate(sql_content.split('\n')):
#     write_to_csv(sql, idx )
    

In [ ]:
import pandas as pd
df = pd.read_csv('local/3.csv', parse_dates=['DATA_DATE'])
df.head(5).to_string()

In [ ]:
from e2b_code_interpreter import Sandbox
import base64
import json
import pandas as pd
from utils.litellm.core import llm
from utils.helper import sql_query_generation_prompt, python_code_generation_prompt
from utils.s3.core import upload_png_to_s3, get_s3_client
from utils.snowflake.core import write_to_csv
chart_data = []
with open('local/charts.json', 'r') as f:
    chart_metadata = json.loads(f.read())
for idx, _ in enumerate(chart_metadata):
    df = write_to_csv(_['SQL'].strip(';'))
    sbx = Sandbox()
    top_5_data = pd.read_csv('local/data.csv').head(5).to_string()
    with open("local/data.csv", "rb") as file:
        sbx.files.write("/home/user/sandbox/data.csv", file)
    result = llm(model='gemini/gemini-2.5-pro-exp-03-25', system_prompt=python_code_generation_prompt, user_prompt=top_5_data, is_json=True)['answer']
    code_to_run = json.loads(result)["code_to_run"] if isinstance(result,str) else result["code_to_run"]
    execution = sbx.run_code(code_to_run)
    print(execution)
    img_bytes = base64.b64decode(execution.results[0].text)
    img_url=upload_png_to_s3(get_s3_client(), 'charts',img_bytes)
    chart_data.append( {'title' : _['Title'], 'description' : _['Description'], 'chart_url': img_url } )

In [ ]:
with open('local/chart_output.json', 'w') as f:
    f.write(json.dumps(chart_data))

In [ ]:
schema = {col: str(df[col].dtype) for col in df.columns}

In [ ]:
code_to_run = """
import subprocess

# Ensure Kaleido is installed
subprocess.run(["pip", "install", "kaleido"], check=True)

import plotly.express as px
import pandas as pd
import io
import base64

df = pd.read_csv('/home/user/sandbox/data.csv')
#NEVER CHANGE THE ABOVE LINES OF CODE, UNLESS EXTRA LIBRARIES ARE REQUIRED FOR INPORT

# Convert DATA_DATE to datetime format
df['DATA_DATE'] = pd.to_datetime(df['DATA_DATE'])

# Create the plot
fig = px.line(df, x='DATA_DATE', y='MOVING_AVERAGE_30', title='S&P 500 30-Day Moving Average')

#NEVER CHANGE THE BELOW LINES OF CODE
img_bytes = io.BytesIO()
fig.write_image(img_bytes, format="png")  # Requires kaleido
img_base64 = base64.b64encode(img_bytes.getvalue()).decode("utf-8")
img_base64
"""


In [ ]:
from utils.sandbox.core import python_sandbox
with open('local/charts.json', 'r') as f:
    chart_metadata = json.loads(f.read())
chart_data = python_sandbox(chart_metadata)

In [1]:
from utils.langgraph.core import entry_point
import json
with open('links.json', 'r', encoding='utf-8') as file:
    links_data = json.load(file)
llm_ready_data = entry_point(links_data)

2025-03-31 19:55:37,592 - =====ENTRY POINT=====
2025-03-31 19:55:37,618 - =====CHART TOOL=====
2025-03-31 19:55:37,640 - =====PREPROCESSING STARTED=====
19:55:37 - LiteLLM:INFO: utils.py:3056 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
2025-03-31 19:55:37,658 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
2025-03-31 19:55:37,666 - Snowflake Connector for Python Version: 3.14.0, Python Version: 3.13.1, Platform: Windows-11-10.0.22631-SP0
2025-03-31 19:55:37,672 - Connecting to GLOBAL Snowflake domain
2025-03-31 19:55:37,674 - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2025-03-31 19:55:39,401 - Request POST https://api.e2b.app/sandboxes
2025-03-31 19:55:40,066 - HTTP Request: POST https://api.e2b.app/sandboxes "HTTP/1.1 201 Created"
2025-03-31 19:55:40,067 - Resp

In [ ]:
from utils.langgraph.core import generate_report_without_streaming
from utils.s3.core import get_s3_client, write_markdown_to_s3
markdown_string =  generate_report_without_streaming(llm_ready_data)
write_markdown_to_s3(get_s3_client(), markdown_string)


20:02:38 - LiteLLM:INFO: utils.py:3056 - 
LiteLLM completion() model= gemini-2.5-pro-exp-03-25; provider = gemini
2025-03-31 20:02:38,397 - 
LiteLLM completion() model= gemini-2.5-pro-exp-03-25; provider = gemini
